# Week 1 Assignment: Quantum Measurement Dataset Foundations

## Task 1 · Environment Setup

### Local virtual environment
- **Windows (Command Prompt):**
  1. `python -m venv quantum_state_tomography`
  2. `quantum_state_tomography\Scripts\activate`
  3. `python -m pip install --upgrade pip wheel`

### Required baseline packages:-
- `pip install qiskit numpy scipy pandas plotly tqdm nbformat`

## Task 2 · Measurement Theory Primer
### Born rule recap
- For a quantum state represented by a density matrix ρ and a measurement operator M_k,
the probability of obtaining outcome k is given by the Born rule: `p(k) = Tr(M_k ρ)`
- In projective measurements, `M_k = P_k` with `P_k^2 = P_k` and `∑_k P_k = I`
- In POVM measurements, `M_k = E_k` where each `E_k` is positive semi-definite and `∑_k E_k = I`.
- For each measurement basis, a numerical completeness check was performed by summing the
measurement operators and verifying that the result equals the identity matrix.

### SIC POVM vs. Pauli projective (single qubit)
- **SIC POVM strengths:-**
    - Informationally complete with fewer outcomes
    - Symmetric structure
    - Resilient to certain noise sources
- **SIC POVM trade-offs:-**
    - Requires careful calibration
    - Uses non-standard measurement bases
    - More complex classical post-processing
- **Pauli projective strengths:-**
    - Uses standard Pauli eigenbases
    - Simple interpretation
    - Widely supported by quantum toolkits
- **Pauli projective trade-offs:-**
    - Requires multiple measurement bases
    - Higher total measurement count

Measurement operators were implemented and serialized using the provided
`build_measurement_model` function.

Pauli projective measurements were selected due to their simplicity, clarity, and ease
of implementation. Although multiple measurement bases are required, they are sufficient
for single-qubit tomography and minimize implementation complexity.

### Reference single-qubit states
The following reference states were prepared using standard single-qubit gates:-
- |0> : Identity (no gate)
- |1> : X gate
- |+> : Hadamard gate
- |-> : X followed by Hadamard
- |+i>: Hadamard followed by S gate


In [1]:
from typing import Dict, Any
import numpy as np
import pathlib

def projector(state: np.ndarray) -> np.ndarray:
    return np.outer(state, state.conj())

def build_measurement_model(config_path: pathlib.Path) -> Dict[str, Any]:
    zero = np.array([1, 0], dtype=complex)
    one  = np.array([0, 1], dtype=complex)

    plus  = (zero + one) / np.sqrt(2)
    minus = (zero - one) / np.sqrt(2)

    plus_i  = (zero + 1j * one) / np.sqrt(2)
    minus_i = (zero - 1j * one) / np.sqrt(2)

    measurements = {
        "Z": {
            "0": projector(zero),
            "1": projector(one),
        },
        "X": {
            "+": projector(plus),
            "-": projector(minus),
        },
        "Y": {
            "+i": projector(plus_i),
            "-i": projector(minus_i),
        }
    }

    I = np.eye(2)
    completeness = {
        basis: np.allclose(sum(ops.values()), I)
        for basis, ops in measurements.items()
    }

    return {
        "type": "Pauli projective",
        "operators": measurements,
        "completeness_check": completeness,
        "notes": "Pauli X/Y/Z projective measurements for single-qubit tomography."
    }


In [2]:
#@title helper functions for density matrix visualization

import numpy as np
import plotly.graph_objects as go
from fractions import Fraction

_CUBE_FACES = (
    (0, 1, 2), (0, 2, 3),  # bottom
    (4, 5, 6), (4, 6, 7),  # top
    (0, 1, 5), (0, 5, 4),
    (1, 2, 6), (1, 6, 5),
    (2, 3, 7), (2, 7, 6),
    (3, 0, 4), (3, 4, 7)
 )

def _phase_to_pi_string(angle_rad: float) -> str:
    """Format a phase angle as a simplified multiple of π."""
    if np.isclose(angle_rad, 0.0):
        return "0"
    multiple = angle_rad / np.pi
    frac = Fraction(multiple).limit_denominator(16)
    numerator = frac.numerator
    denominator = frac.denominator
    sign = "-" if numerator < 0 else ""
    numerator = abs(numerator)
    if denominator == 1:
        magnitude = f"{numerator}" if numerator != 1 else ""
    else:
        magnitude = f"{numerator}/{denominator}"
    return f"{sign}{magnitude}π" if magnitude else f"{sign}π"

def plot_density_matrix_histogram(rho, basis_labels=None, title="Density matrix (|ρ_ij| as bar height, phase as color)"):
    """Render a density matrix as a grid of solid histogram bars with phase coloring."""
    rho = np.asarray(rho)
    if rho.ndim != 2 or rho.shape[0] != rho.shape[1]:
        raise ValueError("rho must be a square matrix")

    dim = rho.shape[0]
    mags = np.abs(rho)
    phases = np.angle(rho)
    x_vals = np.arange(dim)
    y_vals = np.arange(dim)

    if basis_labels is None:
        basis_labels = [str(i) for i in range(dim)]

    meshes = []
    colorbar_added = False
    for i in range(dim):
        for j in range(dim):
            height = mags[i, j]
            phase = phases[i, j]
            x0, x1 = i - 0.45, i + 0.45
            y0, y1 = j - 0.45, j + 0.45
            vertices = (
                (x0, y0, 0.0), (x1, y0, 0.0), (x1, y1, 0.0), (x0, y1, 0.0),
                (x0, y0, height), (x1, y0, height), (x1, y1, height), (x0, y1, height)
            )
            x_coords, y_coords, z_coords = zip(*vertices)
            i_idx, j_idx, k_idx = zip(*_CUBE_FACES)
            phase_pi = _phase_to_pi_string(phase)
            mesh = go.Mesh3d(
                x=x_coords,
                y=y_coords,
                z=z_coords,
                i=i_idx,
                j=j_idx,
                k=k_idx,
                intensity=[phase] * len(vertices),
                colorscale="HSV",
                cmin=-np.pi,
                cmax=np.pi,
                showscale=not colorbar_added,
                colorbar=dict(
                    title="phase ",
                    tickvals=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
                    ticktext=["-π", "-π/2", "0", "π/2", "π"]
                ) if not colorbar_added else None,
                opacity=1.0,
                flatshading=False,
                hovertemplate=
                    f"i={i}, j={j}<br>|ρ_ij|={height:.3f}<br>arg(ρ_ij)={phase_pi}<extra></extra>",
                lighting=dict(ambient=0.6, diffuse=0.7)
            )
            meshes.append(mesh)
            colorbar_added = True

    fig = go.Figure(data=meshes)
    fig.update_layout(
        scene=dict(
            xaxis=dict(
                title="i",
                tickmode="array",
                tickvals=x_vals,
                ticktext=basis_labels
            ),
            yaxis=dict(
                title="j",
                tickmode="array",
                tickvals=y_vals,
                ticktext=basis_labels
            ),
            zaxis=dict(title="|ρ_ij|"),
            aspectratio=dict(x=1, y=1, z=0.7)
        ),
        title=title,
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig.show()


### Visualization helpers
The provided density matrix histogram visualization utility was used to inspect
reconstructed density matrices. The visualization maps magnitude to bar height
and phase to colour.

In [3]:
# Demonstration: random 2-qubit density matrix
dim = 4
A = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
rho = A @ A.conj().T
rho = rho / np.trace(rho)  # normalize

labels = ["00", "01", "10", "11"]
plot_density_matrix_histogram(rho, basis_labels=labels, title="Random 2-qubit state (density matrix)")

In [4]:
#@title helper function Demonstration: canonical Bell states
bell_states = {
    "Φ⁺": np.array([1, 0, 0, 1], dtype=complex) / np.sqrt(2),
    "Φ⁻": np.array([1, 0, 0, -1], dtype=complex) / np.sqrt(2),
    "Ψ⁺": np.array([0, 1, 1, 0], dtype=complex) / np.sqrt(2),
    "Ψ⁻": np.array([0, 1, -1, 0], dtype=complex) / np.sqrt(2)
}

for name, state in bell_states.items():
    density_matrix = np.outer(state, state.conj())
    plot_density_matrix_histogram(
        density_matrix,
        basis_labels=["00", "01", "10", "11"],
        title=f"Bell state {name} (density matrix)"
    )

## Task 3 · QST Data generation
#### Measurement datasets were generated synthetically using finite-shot sampling. Random sampling was performed using NumPy’s multinomial sampler; results reflect finite-shot statistical noise

In [5]:
import numpy as np
from pathlib import Path

# Measurement operators
P0 = np.array([[1, 0], [0, 0]])
P1 = np.array([[0, 0], [0, 1]])

P_plus  = 0.5 * np.array([[1, 1], [1, 1]])
P_minus = 0.5 * np.array([[1, -1], [-1, 1]])

P_plus_i  = 0.5 * np.array([[1, -1j], [1j, 1]])
P_minus_i = 0.5 * np.array([[1, 1j], [-1j, 1]])

def prob(P, rho):
    return np.trace(P @ rho).real

def generate_measurement_data(rho, shots=1000):
    pZ = [prob(P0, rho), prob(P1, rho)]
    pX = [prob(P_plus, rho), prob(P_minus, rho)]
    pY = [prob(P_plus_i, rho), prob(P_minus_i, rho)]

    return {
        "Z": np.random.multinomial(shots, pZ),
        "X": np.random.multinomial(shots, pX),
        "Y": np.random.multinomial(shots, pY),
        "shots": shots
    }

# Reference states
states = {
    "state_0": {
        "label": "|0>",
        "rho": np.array([[1, 0], [0, 0]])
    },
    "state_1": {
        "label": "|1>",
        "rho": np.array([[0, 0], [0, 1]])
    },
    "state_plus": {
        "label": "|+>",
        "rho": np.array([[0.5, 0.5], [0.5, 0.5]])
    }
}

# Save datasets
base = Path("data/single_qubit/measurements")
base.mkdir(parents=True, exist_ok=True)

for fname, info in states.items():
    data = generate_measurement_data(info["rho"])
    data["state_label"] = info["label"]
    np.save(base / f"{fname}.npy", data)

print("Task 3: Measurement data saved successfully.")

Task 3: Measurement data saved successfully.


## Task 4 · Single-Qubit Tomography
#### Single-qubit tomography was performed using linear inversion.

In [6]:
import numpy as np
from pathlib import Path

# Pauli matrices
I  = np.eye(2)
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sz = np.array([[1, 0], [0, -1]])

def expectation(counts):
    return (counts[0] - counts[1]) / np.sum(counts)

Path("data/single_qubit/reconstructions").mkdir(parents=True, exist_ok=True)

for file in Path("data/single_qubit/measurements").glob("*.npy"):
    data = np.load(file, allow_pickle=True).item()

    Ez = expectation(data["Z"])
    Ex = expectation(data["X"])
    Ey = expectation(data["Y"])

    rho_recon = 0.5 * (I + Ex*sx + Ey*sy + Ez*sz)

    np.save(f"data/single_qubit/reconstructions/{file.stem}_recon.npy", rho_recon)

## Task 5 · Validation and Reporting
#### Reconstructed states were validated against ground-truth density matrices using a matrix norm error metric.

In [7]:
import numpy as np
from pathlib import Path

# Ground truth
truth = {
    "state_0": np.array([[1, 0], [0, 0]]),
    "state_1": np.array([[0, 0], [0, 1]]),
    "state_plus": np.array([[0.5, 0.5], [0.5, 0.5]]),
}

print("State | Frobenius error")
print("----------------------")

for file in Path("data/single_qubit/reconstructions").glob("*_recon.npy"):
    state = file.stem.replace("_recon", "")
    rho_hat = np.load(file)

    error = np.linalg.norm(truth[state] - rho_hat)
    print(f"{state:4s} | {error:.4f}")

State | Frobenius error
----------------------
state_0 | 0.0170
state_1 | 0.0300
state_plus | 0.0146
